<a href="https://colab.research.google.com/github/rezlevi/Big-Project/blob/main/weather_forecast.ipynb_fresh" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Függvény a float ellenőrzéshez.

In [ ]:
# -*- coding: utf-8 -*-
from sklearn.metrics import mean_squared_error

def is_float(s): #számok kiszűrése
    try:
        float(s)
        return True
    except ValueError:
        return False


In [ ]:
print(is_float("alma"))
print(is_float("-3.9"))
for i in range(2,9):
  print(i,"páros" if i%2==0 else "páratlan")
for i in ["szilva", "5", "3.14", 3.14]:
  print(i,"float" if is_float(i) else "nem float")


Google drive kezeléséhez import és mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Adatok beolvasása, plusz paraméterek.

In [ ]:
HIST=10
COLS=3
adatok_nyers=[sor.strip().split(sep=";") for sor in open("/content/drive/My Drive/Classroom/Tibor Tajti/neural net application/BP_d.csv")]


In [ ]:
adatok_nyers[:5]

Adatokból kiválasztjuk a felhasználandókat. (avg, max, min hőmérséklet)

In [ ]:
print( len(adatok_nyers), adatok_nyers[0])
adatok=[[float(adat) if is_float(adat) else adat  for adat in sor[1:COLS+1] ] for sor in adatok_nyers[1:]]
print( len(adatok), adatok[0])
print( min([min(sor) for sor in adatok]), max([max(sor) for sor in adatok]))

Az adatokból kiválasztunk tanulási és tesztelési mintahalmazt.

In [ ]:
train=adatok[:20000]
test=adatok[20000:25000]


In [ ]:
train[:5], test[:5]

Az adatsorokból idősoros mintákat állítunk elő.

In [ ]:
import numpy as np
train_x=(np.array([ train[i:i+HIST] for i in range(len(train)-HIST)])-8)/40 # első sortól az ötödikig
train_y=(np.array([ train[i+HIST] for i in range(len(train)-HIST)])-8)/40
#train_x=(np.array([ train[i:i+HIST] for i in range(0,19990)])+35)/40 # első sortól az ötödikig
#train_y=(np.array([ train[i+HIST] for i in range(0,19990)])+35)/40
test_x=(np.array([ test[i:i+HIST] for i in range(len(test)-HIST)])-8)/40 # első sortól az ötödikig
test_y=(np.array([ test[i+HIST] for i in range(len(test)-HIST)])-8)/40

print(train_x.shape, train_y.shape)
print(train_x[0])
print(train_y[0])


Egyszerű futtatás 50 leckével.

In [ ]:
print('train with tensorflow')
import tensorflow as tf
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(HIST,COLS)),
  tf.keras.layers.Dense(100, activation='tanh'),
  tf.keras.layers.Dense(50, activation='tanh'),
  tf.keras.layers.Dense(COLS, activation='tanh')
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

mselst=[]
for _ in range(50):
  model.fit(train_x, train_y, epochs=1)
  y=model.predict(test_x)
  print('prediction MSE:', mean_squared_error(test_y, y))
  print('dummy MSE:', mean_squared_error(test_y[HIST:], test_y[HIST-1:-1]))
  mselst.append(mean_squared_error(test_y, y))


Ugyanaz még egyszer 10 db tanulóval.

In [ ]:
print('train with tensorflow')
import tensorflow as tf
mselstlst=[]
for m in range(10):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(HIST,COLS)),
    tf.keras.layers.Dense(100, activation='tanh'),
    tf.keras.layers.Dense(50, activation='tanh'),
    tf.keras.layers.Dense(COLS, activation='tanh')
  ])

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  mselst=[]
  for epoch in range(50):
    model.fit(train_x, train_y, epochs=1)
    y=model.predict(test_x)
    print(m, epoch, 'prediction MSE:', mean_squared_error(test_y, y))
    print(m, epoch, 'dummy MSE:', mean_squared_error(test_y[HIST:], test_y[HIST-1:-1]))
    mselst.append(mean_squared_error(test_y, y))

  mselstlst.append(mselst)


In [ ]:
print(mselst)
print(mselstlst)
res=np.array(mselstlst)
print(res.shape)
print(np.mean(res))
print(np.mean(res, axis=1))
print(np.mean(res, axis=0))


In [ ]:
from matplotlib import pyplot as plt
plt.plot(np.mean(res, axis=0),"g.")
plt.plot(np.min(res, axis=0),"r.")
plt.plot(np.max(res, axis=0),"b.")
plt.show()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(np.mean(res, axis=0),"g.")
plt.plot(np.min(res, axis=0),"r.")
plt.plot(np.max(res, axis=0),"b.")
plt.show()

In [ ]:
for i in range(len(res)):
  plt.plot(res[i],"g.")
dummy=mean_squared_error(test_y[HIST:], test_y[HIST-1:-1])
plt.plot([dummy for _ in res[0]],"r.")
plt.show()

In [ ]:
train_x2=train_x.reshape((train_x.shape[0], train_x.shape[1]*train_x.shape[2]))
test_x2=test_x.reshape((test_x.shape[0], test_x.shape[1]*test_x.shape[2]))
print(train_x2.shape, test_x2.shape)


In [ ]:
print('train with own simple MLP')

import math, random, time

# available activation functions
def activation_tanh(x):     return math.tanh(x)     # (-1..1)
def dactivation_tanh(x):    return 1.0 - x**2
def activation_sigmoid(x):  return 1.0/(1.0 + math.exp(-x))   # (0..1)
def dactivation_sigmoid(x): return x*(1.0-x)

#acti, dacti = activation_sigmoid, dactivation_sigmoid
acti, dacti = activation_tanh, dactivation_tanh

B = 1
#nn = [2+B, 4, 3, 3]
nn = [len(train_x2[0])+B, 20, 10, len(train_y[0])]
wl=[ [ [random.random()*0.4-0.2 for _ in range(nn[l])] for _ in range(nn[l+1])] for l in range(len(nn)-1)] 
#wl = [ numpy.random.random((nn[l+1], nn[l]))*0.8-0.4 for l in range(len(nn)-1) ]
#wl = [numpy.linspace(-1,1,nn[l]*nn[l+1]).reshape((nn[l+1], nn[l])) for l in range(len(nn)-1)]
#          3
#[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
#                         8
#[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
epoch = 0
sumerr = len(train_x2)
# training
t00=time.time()
while sumerr/len(train_x2)/nn[-1]>=0.0001 and epoch<20:
    sumerr = 0.0
    epoch += 1
    t0=time.time()
    #for inp, out in random.sample(samples, len(samples)):
    for inp, out in zip(train_x2, train_y):
        nl = [ list(inp) + [1.0]*B ]
        for l in range(len(nn)-1):
            nl.append([acti(sum([nl[l][i] * wl[l][j][i] for i in range(nn[l])])) for j in range(nn[l+1])])
            
        error = [out[j] - nl[-1][j] for j in range(nn[-1])]
        delta = [None for _ in range(len(nn)-1)]
        for l in reversed(range(len(nn)-1)):
            if l == len(nn)-2:
                delta[l] = [error[j] * dacti(nl[-1][j]) for j in range(nn[-1])]
            else:
                delta[l] = [sum([delta[l+1][j] * wl[l+1][j][i] for j in range(nn[l+2])])*dacti(nl[l+1][i]) for i in range(nn[l+1])]
            
            for i in range(nn[l]):
                for j in range(nn[l+1]):
                    wl[l][j][i] += 0.01 * delta[l][j] * nl[l][i]

        sumerr += sum( [error[j]**2 for j in range(nn[-1])])
    print (epoch,round(sumerr/len(train_x2)/nn[-1],3), round(time.time()-t0,3))
print (epoch,round(sumerr/len(train_x2)/nn[-1],3), round(time.time()-t00,3))

#testing
y2=[]
#for inp, out in random.sample(samples, len(samples)):
for inp in test_x2:
    nl = [ list(inp) + [1.0]*B ]
    for l in range(len(nn)-1):
        nl.append([acti(sum([nl[l][i] * wl[l][j][i] for i in range(nn[l])])) for j in range(nn[l+1])])
        
    y2.append(nl[-1])
y2=np.array(y2)
print('prediction MSE:', mean_squared_error(test_y, y2))
print('dummy MSE:', mean_squared_error(test_y[1:], test_y[:-1]))




In [ ]:
a=[11,12,13,14,15]
b=[21,22,23,24,25]
[ (a,b) for a,b in zip(a,b) ]

In [ ]:
x=['x1','x2','x3','x4','x5']
y=['y1','y2','y3','y4','y5']
[ (_x,_y) for _x,_y in zip(x,y) ]

In [ ]:
print('train with own simple MLP vectorized')
import random, time
def activation_tanh(x):     return np.tanh(x)     # (-1..1)
def dactivation_tanh(x):    return 1.0 - x**2
def activation_sigmoid(x):  return 1.0/(1.0 + np.exp(-x))   # (0..1)
def dactivation_sigmoid(x): return x*(1.0-x)

acti, dacti = activation_tanh, dactivation_tanh

B = 1
nn = [len(train_x2[0])+B, 100, 50, len(train_y[0])]
wl = [ np.random.random((nn[l+1], nn[l]))*0.2-0.1 for l in range(len(nn)-1)] 
#wl = [numpy.linspace(-1,1,nn[l]*nn[l+1]).reshape((nn[l+1], nn[l])) for l in range(len(nn)-1)]
delta = [np.zeros((nn[l+1])) for l in range(len(nn)-1)]

epoch = 0
sumerr = len(train_x2)
t00=time.time()
while sumerr/len(train_x2)/nn[-1]>=0.0001 and epoch<20:
    sumerr = 0.0
    epoch += 1
    t0=time.time()
    #for inp, out in random.sample(samples, len(samples)):
    #for inp, out in zip(train_x2, train_y):
    for inp, out in random.sample([(a,b) for a,b in zip(train_x2, train_y)],len(train_x2)):
        nl = [ np.array(list(inp) + [1.0]*B) ]
        for l in range(len(nn)-1):
            nl.append(acti(np.dot(wl[l],nl[l])))
        error = out - nl[-1]
        #delta = [None for _ in range(len(nn)-1)]
        for l in reversed(range(len(nn)-1)):
            if l == len(nn)-2:
                #delta[l] = error*dacti(nl[-1])
                delta[l][:] = error*dacti(nl[-1])
            else:
                #delta[l] = numpy.dot(delta[l+1],wl[l+1])*dacti(nl[l+1])
                np.dot(delta[l+1],wl[l+1], out=delta[l])
                delta[l] *= dacti(nl[l+1])
            
            wl[l] += 0.01 * delta[l].reshape((nn[l+1],1))*nl[l].reshape((1,nn[l]))

        sumerr += sum(error**2)
    print (epoch,round(sumerr/len(train_x2)/nn[-1],3), round(time.time()-t0,3))
print (epoch,round(sumerr/len(train_x2)/nn[-1],3), round(time.time()-t00,3))

sumerr = 0.0
y2=[]
for inp in test_x2:
    nl = [ np.array(list(inp) + [1.0]*B) ]
    for l in range(len(nn)-1):
        nl.append(acti(np.dot(wl[l],nl[l])))
    y2.append(nl[-1])
y2=np.array(y2)
print('prediction MSE:', mean_squared_error(test_y, y2))
print('dummy MSE:', mean_squared_error(test_y[1:], test_y[:-1]))


In [ ]:
print('train with own simple MLP vectorized + momentum')
import random, time

def activation_tanh(x):     return np.tanh(x)     # (-1..1)
def dactivation_tanh(x):    return 1.0 - x**2
def activation_sigmoid(x):  return 1.0/(1.0 + np.exp(-x))   # (0..1)
def dactivation_sigmoid(x): return x*(1.0-x)

acti, dacti = activation_tanh, dactivation_tanh

B=1
nn = [len(train_x2[0])+B, 100, 50, len(train_y[0])]
wl = [ np.random.random((nn[l+1], nn[l]))*0.2-0.1 for l in range(len(nn)-1)] 
#wl = [numpy.linspace(-1,1,nn[l]*nn[l+1]).reshape((nn[l+1], nn[l])) for l in range(len(nn)-1)]
delta = [np.zeros((nn[l+1])) for l in range(len(nn)-1)]
cl = [np.zeros_like(_w) for _w in wl]
epoch = 0
sumerr = len(train_x2)
t00=time.time()
while sumerr/len(train_x2)>=0.0001 and epoch<20:
    sumerr = 0.0
    epoch += 1
    t0=time.time()
    #for inp, out in random.sample(samples, len(samples)):
    #for inp, out in zip(train_x2, train_y):
    for inp, out in random.sample([(a,b) for a,b in zip(train_x2, train_y)],len(train_x2)):
        nl = [ np.array(list(inp) + [1.0]*B) ]
        for l in range(len(nn)-1):
            nl.append(acti(np.dot(wl[l],nl[l])))
        error = out - nl[-1]
        #delta = [None for _ in range(len(nn)-1)]
        for l in reversed(range(len(nn)-1)):
            if l == len(nn)-2:
                #delta[l] = error*dacti(nl[-1])
                delta[l][:] = error*dacti(nl[-1])
            else:
                #delta[l] = numpy.dot(delta[l+1],wl[l+1])*dacti(nl[l+1])
                np.dot(delta[l+1],wl[l+1], out=delta[l])
                delta[l] *= dacti(nl[l+1])
            
            wl[l] += 0.2*cl[l]
            cl[l][:]=0.01 * delta[l].reshape((nn[l+1],1))*nl[l].reshape((1,nn[l]))
            wl[l] += 0.8*cl[l]

        sumerr += sum( [error[j]**2 for j in range(nn[-1])])
    print (epoch,round(sumerr/len(train_x2)/nn[-1],3), round(time.time()-t0,3))
print (epoch,round(sumerr/len(train_x2)/nn[-1],3), round(time.time()-t00,3))

sumerr = 0.0
y2=[]
#for inp, out in random.sample(samples, len(samples)):
for inp in test_x2:
    nl = [ np.array(list(inp) + [1.0]*B) ]
    for l in range(len(nn)-1):
        nl.append(acti(np.dot(wl[l],nl[l])))
    y2.append(nl[-1])
y2=np.array(y2)
print('prediction MSE:', mean_squared_error(test_y, y2))
print('stupid MSE:', mean_squared_error(test_y[1:], test_y[:-1]))


In [ ]:
for a, b in zip(test_y, y):
  for _a, _b in zip(a*70-35, b*70-35):
    print(round(_a,2), round(_b,2), end="; ")
  print()

In [ ]:
import numpy as np
import random
def activation_sigmoid(x):  return 1.0/(1.0 + np.exp(-x))   # (0..1)
def dactivation_sigmoid(x): return x*(1.0-x)

def f(x):
    for _x in x:
      y=activation_sigmoid(_x)

x=[random.random()*10-5 for _ in range(100)]
g=np.arange(-16,17)
gg=1.0/(1.0 + np.exp(-g)).astype(np.float64)
gd=gg[1:]-gg[:-1]

def activation_sigmoid_pcw(x):
  b=np.floor(x).astype(np.int32)
  d=x-b
  return gg[b+17]*d+gg[b+16]*(1-d)

def activation_sigmoid_pcw2(x):
  b=np.floor(x).astype(np.int32)
  d=x-b
  return gg[b+16]+gd[b+17]*(1.0-d)

def f2(x):
    for _x in x:
      y=activation_sigmoid_pcw(_x)

In [ ]:
x=np.linspace(-5,5,1000)


In [ ]:
timeit activation_sigmoid(x)


In [ ]:
timeit activation_sigmoid_pcw(x)

In [ ]:
g=np.arange(-16,17)
gg=1.0/(1.0 + np.exp(-g))
x=2.5
b=np.floor(x).astype(np.int32)
print(b)
d=x-b
r=gg[b+17]*d+gg[b+16]*(1-d)
print(r, 1.0/(1.0 + np.exp(-int(x))), 1.0/(1.0 + np.exp(-x)))

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8]

In [ ]:
x=np.linspace(-5,5,100)
y=1.0/(1.0 + np.exp(-x))
y2=activation_sigmoid_pcw(x)
plt.plot(x,y,".")
plt.plot(x,y2,".")
plt.show()